[Data Link: ](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\M.Safnas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import numpy as np
import pandas as pd

In [36]:
df = pd.read_csv('data\IMDB_Dataset.csv')
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [37]:
df.shape

(50000, 2)

In [38]:
df = df.iloc[10000:20000]  # to reduce the size of the dataset for demonstration purposes 
df.head()

,review,sentiment
10000,""" While sporadically engrossing (including a few effectively tender moments) and humorous, the sledgehammer-obvious satire 'Homecoming' hinges on comes off as forced and ultimately unfulfilling. With material like this, timing is everything (Michael M...",negative
10001,"A French novelist, disgusted by his wife's society friends, goes to North Africa for a respite. There he encounters a vivacious & talented Bedouin girl, living in poverty. To spite his wife, who is romancing a Maharajah, he decides to train & educate ...",positive
10002,"It must be remembered that the Gammera movies, like many of the first-series Gozilla films, WERE in fact aimed squarely at kids. Little Kenny and his cohorts are living out the daydreams of the kids in the audience: they get to run around and play wit...",negative
10003,"I gave this film 8 out of 10, reserving 10 for e.g Amadeus, and 9 for Slumdog Millionaire most recently. This film is close to Slumdog, but it is difficult to judge on such film without understanding Balkan life, mentality and a soul which Kusturica p...",positive
10004,"Cheap, gloriously bad cheese from the 80's, the decade of cheese. I watched this one first uncut and un-MST3K'ed, and it was pretty much laugh out loud funny even without the comments.<br /><br />The plot(such as it is) revolves around a post-apocalyp...",negative


In [39]:
df.shape

(10000, 2)

In [41]:
df['review'][10000]

'" While sporadically engrossing (including a few effectively tender moments) and humorous, the sledgehammer-obvious satire \'Homecoming\' hinges on comes off as forced and ultimately unfulfilling. With material like this, timing is everything (Michael Moore knew to release "Fahrenheit 9/11" before the 2004 elections), and the real tragedy of Dante\'s film is that it didn\'t come out 2 years ago, when its message would have carried an energy that would have energized the dissidents further. In 2006, mockery of the well-settled Bush Administration hardly seems as controversially compelling (or imperiled) as it did then."<br /><br />frankly anyone that could be convinced of anything by a ham fisted zombie flick has questionable intelligence. <br /><br />and if you didn\'t notice, michael moore didn\'t exactly help to defeat bush.<br /><br />there was nothing engrossing about this film. i just felt disgust at how blatant and frankly stupid the film was, it was painful to watch. if you are

In [42]:
df['sentiment'].value_counts()

sentiment
negative    5125
positive    4875
Name: count, dtype: int64

In [43]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [44]:
df.duplicated().sum()

np.int64(22)

In [45]:
df.drop_duplicates(inplace=True)

In [46]:
df.duplicated().sum()

np.int64(0)

# Basic Preprocessing
  - Remove tags - HTML
  - Lower case
  - remove stopwords

In [65]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [63]:
df['review'] = df['review'].apply(remove_tags)

In [64]:
df.head()

,review,sentiment
10000,""" sporadically engrossing (including effectively tender moments) humorous, sledgehammer-obvious satire 'homecoming' hinges comes forced ultimately unfulfilling. material like this, timing everything (michael moore knew release ""fahrenheit 9/11"" 2004 e...",negative
10001,"french novelist, disgusted wife's society friends, goes north africa respite. encounters vivacious & talented bedouin girl, living poverty. spite wife, romancing maharajah, decides train & educate girl, present parisian society princesse tam tam...the...",positive
10002,"must remembered gammera movies, like many first-series gozilla films, fact aimed squarely kids. little kenny cohorts living daydreams kids audience: get run around play top-secret stuff adults stand allow it; get cavort monsters, even bad guys enter, ...",negative
10003,"gave film 8 10, reserving 10 e.g amadeus, 9 slumdog millionaire recently. film close slumdog, difficult judge film without understanding balkan life, mentality soul kusturica presents masterfully. understand really need one balkan. amazing movie, much...",positive
10004,"cheap, gloriously bad cheese 80's, decade cheese. watched one first uncut un-mst3k'ed, pretty much laugh loud funny even without comments.the plot(such is) revolves around post-apocalyptic world ai robots revolted(sound familiar?) destroyed pretty muc...",negative


In [51]:
df['review'][10000]

'" While sporadically engrossing (including a few effectively tender moments) and humorous, the sledgehammer-obvious satire \'Homecoming\' hinges on comes off as forced and ultimately unfulfilling. With material like this, timing is everything (Michael Moore knew to release "Fahrenheit 9/11" before the 2004 elections), and the real tragedy of Dante\'s film is that it didn\'t come out 2 years ago, when its message would have carried an energy that would have energized the dissidents further. In 2006, mockery of the well-settled Bush Administration hardly seems as controversially compelling (or imperiled) as it did then."frankly anyone that could be convinced of anything by a ham fisted zombie flick has questionable intelligence. and if you didn\'t notice, michael moore didn\'t exactly help to defeat bush.there was nothing engrossing about this film. i just felt disgust at how blatant and frankly stupid the film was, it was painful to watch. if you are going to do something like this you

In [52]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [53]:
df['review'][10000]

'" while sporadically engrossing (including a few effectively tender moments) and humorous, the sledgehammer-obvious satire \'homecoming\' hinges on comes off as forced and ultimately unfulfilling. with material like this, timing is everything (michael moore knew to release "fahrenheit 9/11" before the 2004 elections), and the real tragedy of dante\'s film is that it didn\'t come out 2 years ago, when its message would have carried an energy that would have energized the dissidents further. in 2006, mockery of the well-settled bush administration hardly seems as controversially compelling (or imperiled) as it did then."frankly anyone that could be convinced of anything by a ham fisted zombie flick has questionable intelligence. and if you didn\'t notice, michael moore didn\'t exactly help to defeat bush.there was nothing engrossing about this film. i just felt disgust at how blatant and frankly stupid the film was, it was painful to watch. if you are going to do something like this you

In [54]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\M.Safnas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
df['review'][10000]

'" sporadically engrossing (including effectively tender moments) humorous, sledgehammer-obvious satire \'homecoming\' hinges comes forced ultimately unfulfilling. material like this, timing everything (michael moore knew release "fahrenheit 9/11" 2004 elections), real tragedy dante\'s film come 2 years ago, message would carried energy would energized dissidents further. 2006, mockery well-settled bush administration hardly seems controversially compelling (or imperiled) then."frankly anyone could convinced anything ham fisted zombie flick questionable intelligence. notice, michael moore exactly help defeat bush.there nothing engrossing film. felt disgust blatant frankly stupid film was, painful watch. going something like need bit wit. sadly none. poorly done satire actually opposite intended effect. say, friends like needs enemies.'

In [56]:
df.head()

,review,sentiment
10000,""" sporadically engrossing (including effectively tender moments) humorous, sledgehammer-obvious satire 'homecoming' hinges comes forced ultimately unfulfilling. material like this, timing everything (michael moore knew release ""fahrenheit 9/11"" 2004 e...",negative
10001,"french novelist, disgusted wife's society friends, goes north africa respite. encounters vivacious & talented bedouin girl, living poverty. spite wife, romancing maharajah, decides train & educate girl, present parisian society princesse tam tam...the...",positive
10002,"must remembered gammera movies, like many first-series gozilla films, fact aimed squarely kids. little kenny cohorts living daydreams kids audience: get run around play top-secret stuff adults stand allow it; get cavort monsters, even bad guys enter, ...",negative
10003,"gave film 8 10, reserving 10 e.g amadeus, 9 slumdog millionaire recently. film close slumdog, difficult judge film without understanding balkan life, mentality soul kusturica presents masterfully. understand really need one balkan. amazing movie, much...",positive
10004,"cheap, gloriously bad cheese 80's, decade cheese. watched one first uncut un-mst3k'ed, pretty much laugh loud funny even without comments.the plot(such is) revolves around post-apocalyptic world ai robots revolted(sound familiar?) destroyed pretty muc...",negative


In [57]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [58]:
X.head()

,review
10000,""" sporadically engrossing (including effectively tender moments) humorous, sledgehammer-obvious satire 'homecoming' hinges comes forced ultimately unfulfilling. material like this, timing everything (michael moore knew release ""fahrenheit 9/11"" 2004 e..."
10001,"french novelist, disgusted wife's society friends, goes north africa respite. encounters vivacious & talented bedouin girl, living poverty. spite wife, romancing maharajah, decides train & educate girl, present parisian society princesse tam tam...the..."
10002,"must remembered gammera movies, like many first-series gozilla films, fact aimed squarely kids. little kenny cohorts living daydreams kids audience: get run around play top-secret stuff adults stand allow it; get cavort monsters, even bad guys enter, ..."
10003,"gave film 8 10, reserving 10 e.g amadeus, 9 slumdog millionaire recently. film close slumdog, difficult judge film without understanding balkan life, mentality soul kusturica presents masterfully. understand really need one balkan. amazing movie, much..."
10004,"cheap, gloriously bad cheese 80's, decade cheese. watched one first uncut un-mst3k'ed, pretty much laugh loud funny even without comments.the plot(such is) revolves around post-apocalyptic world ai robots revolted(sound familiar?) destroyed pretty muc..."


In [59]:
y

10000    negative
10001    positive
10002    negative
10003    positive
10004    negative
10005    negative
10006    negative
10007    negative
10008    negative
10009    positive
10010    positive
10011    negative
10012    positive
10013    positive
10014    negative
10015    positive
10016    negative
10017    positive
10018    positive
10019    positive
10020    negative
10021    negative
10022    positive
10023    positive
10024    negative
10025    negative
10026    negative
10027    negative
10028    negative
10029    negative
10030    negative
10031    positive
10032    positive
10033    negative
10034    negative
10035    negative
10036    positive
10037    negative
10038    negative
10039    negative
10040    positive
10041    positive
10042    positive
10043    positive
10044    negative
10045    positive
10046    positive
10047    negative
10048    negative
10049    negative
10050    positive
10051    negative
10052    positive
10053    positive
10054    positive
10055    p

In [60]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [61]:
y

array([0, 1, 0, ..., 1, 1, 0], shape=(9978,))

In [66]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [67]:
X_train.shape

(7982, 1)

In [68]:
X_test.shape

(1996, 1)

In [69]:
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [70]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [71]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(7982, 47702))

In [76]:
X_test_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1996, 47702))

## model-1

In [72]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

,"priors priors: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None
,"var_smoothing var_smoothing: float, default=1e-9Portion of the largest variance of all features that is added tovariances for calculation stability... versionadded:: 0.20",1e-09


In [73]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6347695390781564

In [77]:
confusion_matrix(y_test,y_pred)

array([[862, 163],
       [159, 812]])

## model-2

In [75]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)

y_pred = rf.predict(X_test_bow)

accuracy_score(y_test,y_pred)

0.8386773547094188

### method-01

## BoW

In [78]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8351703406813628

### method-02

# N grams

In [79]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.841182364729459

## TFIDF

### method-03

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [81]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [82]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8401803607214429